# `Python Webscraping Indeed`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# Purpose & Outcome:

+ `Webscrape Indeed`: take position, job title,location,date of posting, string of qualifications

+ Use a list of words related to your skills or skills you are interested in and extract from job post qualifications section.

+ Date time formating

`------------------------------`

# `Next video Word Cloud and NLP using these data`


In [1]:
# pip3 install wordcloud # (if you don't have already)
import pandas as pd
import numpy as np
import PIL              # for images
import wordcloud        # ploting word cloud
import requests         # grab web-page
import pickle           # save file
from bs4 import BeautifulSoup as bsopa  # parse web-page
import datetime         # format date/time
from collections import defaultdict
import re               # regular expressions

In [2]:
# if you don't have wordcloud, get that now:

# !pip3 install wordcloud
# conda install -c conda-forge wordcloud

In [3]:
'''
ex.) https://www.indeed.com/jobs?q=data+scientist&l=california&start=10

range(0,150,10): each page will have "start=0,start=10,start=20 etc" which deals with
going through each page, but not exactly 10 entries/pg. 

string formatting is used to denote what job and location we want: you can use a string
separated by space and it will be interpreted by the website. 

sou=bsopa(y.text,'lxml') is taking our get request and converting to text in the format
of 'lxml' but we can replace with 'html.parser' as well.

Each job post can be parsed by the 'div',{"class":"jobsearch-SerpJobCard"} or 
'div', {'class': 'row'} depending on how you want to search

After that we get each piece of information we want to obtain: job title, location etc.

the only difficult and frustrating part is getting all the raw text for each posting
relating to the qulifications. We have to open a link, iterate through it and then extract
all the information with a try/except block and then further process.
'''

gg=[]
for j in range(0,150,10):
    position,location='data scientist','california'
    y=requests.get('https://www.indeed.com/jobs?q={}&l={}&sort=date='.format(position,location)+str(j))

    # y=requests.get('https://www.indeed.com/jobs?q=data+scientist&l=california&sort=date='+str(i))
    sou=bsopa(y.text,'lxml')

#     for ii in sou.find_all('div', {'class': 'row'}):
    for ii in sou.find_all('div',{"class":"jobsearch-SerpJobCard"}):

        job_title = ii.find('a', {'data-tn-element': 'jobTitle'})['title']
        company_name = ii.find('span', {'class': 'company'}).text.strip()    
        location=ii.find('span',{"class":"location"})
        post_date = ii.find('span', attrs={'class': 'date'})
        summary=ii.find('div',attrs={'class':'summary'})

        if location:
            location=location.text.strip()
        else:
            location=ii.find('div',{"class":"location"})
            location=location.text.strip()

        k=ii.find('h2', {'class':"title"})
        p=k.find(href=True)
        v=p['href']
        f_=str(v).replace('&amp;','&') # links to iterate for qualification text
        
        
        datum = {'job_title': job_title,
                    'company_name': company_name,
                    'location': location,
                    'summary':summary.text.strip(),
                    'post_Date':post_date.text,
                    'Qualification_link': f_}

#         gg.append([location,job_title,company_name,post_date.text,summary.text.strip()
#                   ,f_]) 
        gg.append(datum)


In [18]:
len(gg)
gg[0]

{'job_title': 'Data Scientist, Medical Diagnostics',
 'company_name': 'Specific Diagnostics',
 'location': 'Mountain View, CA 94043',
 'summary': 'The development of data-driven visualization tools for the explorative analysis of high-dimensional data.\nUsed for bloodstream infection Specific’s solution…',
 'post_Date': '30+ days ago',
 'Qualification_link': '/pagead/clk?mo=r&ad=-6NYlbfkN0ASXGwdLWjBNYivRaShgyLT0X0BdS_EVSQWni2gtHz1pazciS9pJg_LVuqiSysEs-TMIwFFOo7W9bIb5VzrANtYopVIA4KVaF9yg6fI6EvZwtW2vAGjQWCOF3vWYBeKjokeHeoKgHupsPPJigSH8DaOChlEr7xUj3CXaKRa_EOHrA6Ldh3t0mnVz60SF4_2_jhFn1u1wbdVtozdppWb5GiEKmniDeQACKzNRIGkP3xmVqUnjq5MMrmY1NqR7k5fh3cPGaATLJsZd-WryLFGCzYyZ0O0DSDpMoFrVmXLdHTYbc2rlN4D-os2fc8yh2hY3l-w1BBYD4ni2Xijf6AbsJAwgc-TJ7zuUfwXogqDiLRBm7Myr-kwZKeXDZ9VLJdEiUmvYej_PpYSXrLTEZWMpGbI04d6fAr384U39vUPc-x5r-XOXRxt9hBKEZJUol9Q3zlu6xQNTvceFzUiPVRh_jQd1ulw5_cTkvDGmRIMNJG9lg==&p=0&fvj=1&vjs=3'}

# `Links to extract qualification information`

+ `This will be used to take the links used for when you click a job and it expands the small text`

+ `After you exand the text it will show the full job description and relative information`


In [5]:
# get all qualification page text: key=index, value=string of text for qualification
hoop=[]
for i in range(len(gg)):
    op=requests.get('https://www.indeed.com'+gg[i]['Qualification_link'])
    sou_=bsopa(op.text,'html.parser')
    for ii in sou_.find('div',{'class':'jobsearch-jobDescriptionText'}):
        try:
            hoop.append([i,''.join(ii.text.strip())])
        except AttributeError:
            hoop.append([i,''])
hoop[0]

[0,
 'The CompanyThe world is facing a medical crisis, bacteria are increasingly evolving resistance to even our strongest antibiotics. The problem is already very real and immediate; for example, bloodstream infection leading to sepsis is now responsible for more than half of all deaths in hospitals and is the most expensive condition treated in hospitals. Sepsis mortality rate increases >6% every hour without effective antibiotic treatment. Yet, despite the life and death urgency, and healthcare cost impacts, current methodologies require 3 days to determine the correct antibiotic.']

In [21]:
# create dictionary with values as lists
dct_lst= defaultdict(list)
for i in hoop:
    dct_lst[i[0]].append(i[1])
u=[]
for i in dct_lst.values(): # string join: lists of lists of strings
    u.append(''.join(i))


In [31]:
# one entry of our qualification text:
u[2]

"Our Mission\nWe’re here to create a safer, happier and more mindful future for all with the help of data science, engineering, design, and mobile technology. We're starting by reinventing insurance, by rethinking the technologies that enable it, but our true goal is to build a platform that rewards people for driving well — creating safer roads with fewer accidents in the process.\n\nBacked by impressive funding, we're poised to re-engineer a trillion-dollar category. We’re using rich customer insights, advanced technology and data science to build our cloud-native InsurTech solution. We're out to change behavior and promote mindful living.\n\nJob Requirements\nYou must have all the skills and experiences associated with one of the options below. These options change over time and reflect the current needs of the team.\nOption one - Telematics ML\nMachine learning: Kaggle grandmaster-level skills with history of deploying models to production\nTelematics experience: You have led telem

In [43]:
jobs_=pd.concat([pd.DataFrame(gg),pd.DataFrame(u,columns=['Qual_Text'])],axis=1)
jobs_.head()


,job_title,company_name,location,summary,post_Date,Qualification_link,Qual_Text
0,"Data Scientist, Medical Diagnostics",Specific Diagnostics,"Mountain View, CA 94043",The development of data-driven visualization t...,30+ days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0ASXGwdLWjBNYivRa...,The CompanyThe world is facing a medical crisi...
1,Data Scientist,Laxmi Therapeutic Devices,"Goleta, CA 93117",7+ years' practical experience manipulating da...,16 days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0ALgD31io3l0I0Y-r...,Data ScientistLaxmi Therapeutic Devices – Gole...
2,Data Scientist,Blue Owl,"San Francisco, CA","We’re using rich customer insights, advanced t...",30+ days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0D3UvD5kBSgX9r9tF...,"Our Mission\nWe’re here to create a safer, hap..."
3,Data Engineer,"Amick Brown, LLC","Sunnyvale, CA",Develops technical tools and programming that ...,Today,/pagead/clk?mo=r&ad=-6NYlbfkN0A74pTrSPrBtiJlYH...,"Data EngineerSunnyvale, CAAmick Brown is seeki..."
4,Data Scientist,Triplebyte,California,You'll report directly to Triplebytes' Head of...,22 days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0AMr11YIOo206dX9C...,About Triplebyte\n\nTriplebyte is transforming...


# `Parse posting date`: convert string data to official dates

In [37]:
import re
v=[]
for i in jobs_['post_Date']:

    if re.findall(r'[0-9]',i):
        # if the string has digits convert each entry to single string: ['3','0']->'30'
        b=''.join(re.findall(r'[0-9]',i))
        
        # convert string int to int and subtract from today's date and format
        g=(datetime.datetime.today()-datetime.timedelta(int(b))).strftime('%m-%d-%Y')

        v.append(g)
        
    else: # this will contain strings like: 'just posted' or 'today' etc before convert
        v.append(datetime.datetime.today().strftime('%m-%d-%Y'))
v[:5]


['09-10-2020', '09-24-2020', '09-10-2020', '10-10-2020', '09-18-2020']

In [42]:
# fixed posting date to date format instead of string: last column
jobs_['posting_date_fixed']=v
jobs_.head()

,job_title,company_name,location,summary,post_Date,Qualification_link,Qual_Text,posting_date_fixed,skill_matches
0,"Data Scientist, Medical Diagnostics",Specific Diagnostics,"Mountain View, CA 94043",The development of data-driven visualization t...,30+ days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0ASXGwdLWjBNYivRa...,The CompanyThe world is facing a medical crisi...,09-10-2020,"[python, sql, visualization]"
1,Data Scientist,Laxmi Therapeutic Devices,"Goleta, CA 93117",7+ years' practical experience manipulating da...,16 days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0ALgD31io3l0I0Y-r...,Data ScientistLaxmi Therapeutic Devices – Gole...,09-24-2020,"[python, sql, statistics, algorithms]"
2,Data Scientist,Blue Owl,"San Francisco, CA","We’re using rich customer insights, advanced t...",30+ days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0D3UvD5kBSgX9r9tF...,"Our Mission\nWe’re here to create a safer, hap...",09-10-2020,"[python, machine learning, deep learning, pandas]"
3,Data Engineer,"Amick Brown, LLC","Sunnyvale, CA",Develops technical tools and programming that ...,Today,/pagead/clk?mo=r&ad=-6NYlbfkN0A74pTrSPrBtiJlYH...,"Data EngineerSunnyvale, CAAmick Brown is seeki...",10-10-2020,"[python, sql, aws, machine learning]"
4,Data Scientist,Triplebyte,California,You'll report directly to Triplebytes' Head of...,22 days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0AMr11YIOo206dX9C...,About Triplebyte\n\nTriplebyte is transforming...,09-18-2020,"[aws, machine learning]"




# `Create a list of skills that you may have or general list`

+ This can be used in a few ways, for example if I put all my skills and see what matched up for jobs.

+ We could generate a corpus and start parsing through job postings not just data scientist, but similar jobs as well. After all the matches for indeed are not the best!

In [2]:
buzz_words=['Python','SQL','AWS', 'Machine learning','Deep learning','Text mining',
'NLP','SAS','Tableau','Sagemaker','Tensorflow','Spark', 'numpy', 'MongDB','PSQL',
"Postgres", 'Pandas', 'RESTFUL','NLP','Statistics','Algorithms','Visualization',
'GCP','Google Cloud','Naive Bayes','Random Forest','Bachelors degree','Masters degree'
'Java','Pyspark','Postgres','MySQL','Github','Docker','Machine Learning','C+',
'C++','Pytorch','Jupyter Notebook','R Studio','R-Studio','Forecasting','Hive',
'PhD','GCP','Numpy','NoSQL','Neo4j','Neural Network','Clustering','Linear Algebra',
'Google Colab','Data Mining','Regression','Time Series','ETL','Data Wrangling',
'Web Scraping','Feature Extraction','Featuring Engineering','Scipy','ML','DL']
buzz_words_list=[x.lower() for x in buzz_words]  # convert list to lowercase to parse

yo=[]
for i in range(len(jobs_.Qual_Text)):
    a=buzz_words_list
    dd=[x for x in a if x in jobs_.Qual_Text[i].lower()]
    yo.append(dd)
jobs_['skill_matches']=yo
jobs_.head(7)


In [65]:

filename='Indeed_scrape_Oct2020.txt'
file=open(filename,'wb')
pickle.dump(jobs_,file)

file_ =open(filename,'rb')
new_file_ =pickle.load(file_)
new_file_.head(10)

,job_title,company_name,location,summary,post_Date,Qualification_link,Qual_Text,skill_matches
0,"Data Scientist, Medical Diagnostics",Specific Diagnostics,"Mountain View, CA 94043",The development of data-driven visualization t...,30+ days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0ASXGwdLWjBNYivRa...,The CompanyThe world is facing a medical crisi...,"[python, sql, visualization, c+, c++, nosql]"
1,Data Scientist,Laxmi Therapeutic Devices,"Goleta, CA 93117",7+ years' practical experience manipulating da...,16 days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0ALgD31io3l0I0Y-r...,Data ScientistLaxmi Therapeutic Devices – Gole...,"[python, sql, statistics, algorithms]"
2,Data Scientist,Blue Owl,"San Francisco, CA","We’re using rich customer insights, advanced t...",30+ days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0D3UvD5kBSgX9r9tF...,"Our Mission\nWe’re here to create a safer, hap...","[python, machine learning, deep learning, pand..."
3,Data Engineer,"Amick Brown, LLC","Sunnyvale, CA",Develops technical tools and programming that ...,Today,/pagead/clk?mo=r&ad=-6NYlbfkN0A74pTrSPrBtiJlYH...,"Data EngineerSunnyvale, CAAmick Brown is seeki...","[python, sql, aws, machine learning, machine l..."
4,Data Scientist,Triplebyte,California,You'll report directly to Triplebytes' Head of...,22 days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0AMr11YIOo206dX9C...,About Triplebyte\n\nTriplebyte is transforming...,"[aws, machine learning, machine learning, time..."
5,"Data Scientist, Analytics",Evernote,"Redwood City, CA 94063 (Downtown area)","Apply your expertise in quantitative analysis,...",3 days ago,/rc/clk?jk=e6e04f20bd7ac8f1&fccid=4d2449c755ba...,About the team:\n\nThe Evernote Analytics team...,"[python, sql, sas, tableau, statistics, visual..."
6,Data Scientist,Apple,"Santa Clara Valley, CA 95014","In-depth knowledge of digital analytics data, ...",Today,/rc/clk?jk=5a488b6a19d296c3&fccid=c1099851e979...,"Summary\nPosted: Oct 9, 2020\nWeekly Hours: 40...","[sql, tableau, visualization]"
7,Data Scientist,"RMDS Lab, Inc","South Lake, CA",Provide consultation on data science and machi...,17 days ago,/company/RMDS-Lab/jobs/Data-Scientist-c487fd99...,About RMDS LabRMDS Lab and its Global Associat...,"[python, machine learning, machine learning]"
8,Data Scientist 2,Xoom,"San Francisco, CA 94105 (Financial District area)",Clear subject matter expert who can help mento...,Today,/rc/clk?jk=00c2ef3c615076e9&fccid=978d9fd9799d...,Who we are: Fueled by a fundamental belief tha...,[statistics]
9,Data Science Intern,Parsec Education,"Fresno, CA 93721 (Central area)",Parsec Education specializes in analyzing stat...,10 days ago,/pagead/clk?mo=r&ad=-6NYlbfkN0B5bikR7eyU0Vk4cd...,Greetings Applicant!Here at Parsec Education w...,"[python, sql, machine learning, visualization,..."


# <font color=red>LIKE</font>, Share &

# <font color=red>SUB</font>scribe

# `Next Video: Word Cloud From Indeed Job Postings, with NLP`

`--------------------`

https://www.jobspikr.com/blog/scraping-indeed-job-data-using-python/

https://jlgamez.com/how-i-scrape-jobs-data-from-indeed-com-with-python/

http://rstudio-pubs-static.s3.amazonaws.com/495949_1ab68b7e55cf4a3ab1fc00285e28fbcc.html

https://chrislovejoy.me/job-scraper/

https://www.youtube.com/watch?v=eN_3d4JrL_w